
<!-- For PDF export only -->
::: {pdf}
## Assignment.
Import, tidy and analyze the NYPD Shooting Incident dataset obtained. Be sure your project is reproducible and contains some visualization and analysis. You may use the data to do any analysis that is of interest to you. You should include at least two visualizations and one model. Be sure to identify any bias possible in the data and in your analysis.
:::

In [ ]:
library(tidyverse)
library(lubridate)
library(forecast)

In [2]:
# import the source data and put it in a df
source_url <- paste0(
  "https://data.cityofnewyork.us/api/views/833y-fsy8/rows.csv?",
  "accessType=DOWNLOAD"
)

incident_df <- read.csv(source_url)

## Explore

In [ ]:
# see what columns we have and what data types
glimpse(incident_df)

I already see lots of nulls, empty strings, missing values, etc. Let's take a closer look at some of the categorical columns to see if there are a limited number of consistently entered values or if they were entered as free text, which might be too difficult to clean.

In [ ]:
# break out all the unique values and counts
desc_counts <- lapply(incident_df[,
  c(
    "LOC_CLASSFCTN_DESC",
    "LOCATION_DESC",
    "PERP_RACE",
    "VIC_RACE",
    "LOC_OF_OCCUR_DESC",
    "VIC_SEX",
    "PERP_SEX"
  )
], table)

print(desc_counts)

Everything seems to be consistently entered (no misspellings or variations.) But there is a weird mix of "unknown", "U", and "null". It will probably be best to recode empty values as "Unknown" for consistency. There is something weird in a few columns too.

In [ ]:
unique(incident_df$PERP_RACE)
table(incident_df$PERP_RACE)


Oh, that's annoying - there is an empty string ```''``` as one of the largest groups, I guess the best option will be to categorize that as ```"UNKNOWN"``` as well. While I'm at it I'm going to make the date time columns a little more usable by separating out the date and time and converting them to the right type.


---

## Cleanup

In [ ]:
clean_incident_df <- incident_df %>%
  mutate(
    # Combine the date and time into a proper DateTime object
    Date = as.POSIXct(
      paste(OCCUR_DATE, OCCUR_TIME), 
      format = "%m/%d/%Y %H:%M:%S"
    )
  ) %>%
  rename(
    In_Out = LOC_OF_OCCUR_DESC, 
    Location_Category = LOC_CLASSFCTN_DESC, 
    Location_details = LOCATION_DESC
  ) %>%
  select(
    Date, BORO, Location_Category, Location_details, 
    In_Out, OCCUR_DATE, OCCUR_TIME, 
    -JURISDICTION_CODE, -X_COORD_CD, -Y_COORD_CD,
    -Latitude, -Longitude, -Lon_Lat, -PRECINCT,
    everything()
  ) %>%
  mutate(
    # Recode specific values in PERP_RACE and VIC_RACE
    PERP_RACE = recode(PERP_RACE, 
                       "ASIAN / PACIFIC ISLANDER" = "ASIAN_PAC_ISLAND",
                       "AMERICAN INDIAN/ALASKAN NATIVE" = "AM_INDIAN/ALASKAN"
    ),
    VIC_RACE = recode(VIC_RACE, 
                      "ASIAN / PACIFIC ISLANDER" = "ASIAN_PAC_ISLAND",
                      "AMERICAN INDIAN/ALASKAN NATIVE" = "AM_INDIAN/ALASKAN"
    ),
    # Recode empty or null values to "UNKNOWN" for PERP_RACE
    PERP_RACE = ifelse(PERP_RACE == "", "UNKNOWN", PERP_RACE),
    
    # Recode unknown, empty, or NA values to "U" for both victim and perpetrator sex
    PERP_SEX = ifelse(PERP_SEX %in% c("Unknown", "","(null)", NA), "U", PERP_SEX),
    VIC_SEX = ifelse(VIC_SEX %in% c("Unknown", "", NA), "U", VIC_SEX)
  )

# Check that the recoding worked as expected
unique(clean_incident_df$PERP_SEX)  # Should show only "M", "F", and "U"
unique(clean_incident_df$VIC_SEX)   # Should show only "M", "F", and "U"
# check that I have the columns and order that I wanted
glimpse(clean_incident_df)

# check that we fixed the empty string values
unique(clean_incident_df$PERP_RACE)

# printing a df is a little uglier in some ways but prevents text
# overlap when there are lots of columns or long column names
print(tail(clean_incident_df))

I'm going to make a few different dataframes with different groups for eventual analysis and plotting. Things I'm going to start with

* Daily incidents over time to look for general trends
* Incidents by month and year
* Incidents by borough
* Incidents by month (not over time, so total incidents that occured in each month summed over all years)
* Time and year data broken down by borough
* Victim and perpetrator by sex

In [ ]:
# for plotting incidents over time
time_series_df <- clean_incident_df %>%
  mutate(simple_date = as.Date(OCCUR_DATE, format = "%m/%d/%Y")) %>%
  group_by(simple_date) %>%
# Add a new column that represents only the month and year
# This step may be unnecessary since I have a good date column
# but it's easier for me to understand
  summarise(total_by_day = n()) %>%
  mutate(month_year = floor_date(simple_date, "month")) 

# for plotting overtime by month and year 
df_aggregated <- time_series_df %>%
  mutate(year = format(simple_date, "%Y"),
         month = format(simple_date, "%m")) %>%
  group_by(year, month) %>%
  summarise(total_by_month = sum(total_by_day)) %>%
   mutate(
    year = as.numeric(year),   # Convert year to numeric
    month = as.numeric(month)  # Convert month to numeric
  ) %>%
  ungroup()

# borough totals
total_by_borough <- clean_incident_df %>%
  group_by(BORO) %>%
  summarize(total_incidents = n())

# monthly borough totals
monthly_totals_by_borough <- clean_incident_df %>%
  mutate(month = floor_date(Date, "month")) %>%
  mutate(month = as.Date(month)) %>%
  group_by(BORO, month) %>%
  summarize(monthly_incidents = n()) %>%
  ungroup()

# victim and perp by sex
totals_by_sex <- clean_incident_df %>%
  group_by(PERP_SEX, VIC_SEX) %>%
  summarise(
    Total_Victims = n(),        # Count number of victims in each group
    Total_Perps = n()           # Count number of perpetrators in each group
  )


tail(time_series_df)
tail(df_aggregated)
tail(total_by_borough)
tail(monthly_totals_by_borough)
tail(totals_by_sex,9)

That's looks pretty good. We'll progress through some basic plots to see what patterns or trends we see which can guide the next visualizations.

---

## Visualization


In [8]:
# Set global plot size options for the notebook
options(repr.plot.width = 10, repr.plot.height = 7)

In [ ]:
#| fig-align: center
theme_set(theme_minimal())

# plot daily incidents
ggplot(time_series_df, aes(x = simple_date, y = total_by_day)) +
  geom_line(color = "skyblue") +
  labs(
    title = "Fig.1 - Daily Incidents",
    x = "Date",
    y = "Incident Count"
  )

Fig 1 -  Daily incidents - hard to see many trends since the data is so noisy, but it does look like it's generally periodic and there is a big spike around the first covid summer.

---


In [ ]:
#| fig-align: center

# plot monthly incidents over time
ggplot(time_series_df, aes(x = month_year, y = total_by_day)) +
  geom_bar(stat = "identity", fill = "skyblue") +
  labs(
       title = "Fig. 2 - Total Occurrences by Month",
       x = "Year", 
       y = "Total Occurrences",
       caption = "Source: NYPD Shooting Incident Data, 2020") +
  scale_x_date(date_labels = "%Y", date_breaks = "1 year")


Fig 2 -  Total occurences by month over time - clearer seasonality and a little easier to see the pre- and post-covid trends

---

In [ ]:
#| fig-align: center
# plot yearly incidents
ggplot(time_series_df, aes(
  x = year(simple_date), 
  y = total_by_day)) +
  geom_bar(stat = "identity", fill = "skyblue") +
  labs(
       title = "Fig. 3 - Total Occurrences by Year",
       x = "Year",
       y = "Total Occurrences")



Fig 3 -  Total occurences by year - now we can see trends. Decreasing incidents starting in the early 20-teens and flattening out, before a big covid spike and almost back down to pre-covid levels.


In [ ]:

# Plot occurrences by month (across all years)
ggplot(time_series_df, aes(
  x = month(simple_date, label = TRUE),
  y = total_by_day
  )) +
  geom_bar(
    stat = "identity", 
    fill = "skyblue"
  ) +
  labs(
    title = "Fig. 4 - Total Occurrences by Month",
    x = "Month",
    y = "Total Occurrences"
  )



Fig 4 -  Total occurences by month - pretty strong visual trend towards higher incidents in the hottest months, which is a well studied phenomenon.

---


In [ ]:

# Plot total by borough
ggplot(total_by_borough, aes(
  x = BORO,
  y = total_incidents
)) +
  geom_bar(
    stat = "identity",
    fill = "skyblue"
  ) +
  labs(
    title = "Fig. 5 - Total by Borough",
    x = "Borough",
    y = "Total"
  )


Fig 5 -  Total by borough - generaly interesting, but would be more useful with the context of per capita and per area data for the boroughs.

---


In [ ]:

# Borough totals monthly
ggplot(monthly_totals_by_borough, aes(
  x = month,
  y = monthly_incidents,
  color = BORO)
  ) +
  # I wanted to try a line instead of a bar
  geom_line(linewidth = 1.2) +
  geom_point(size = 2) +
  labs(
    title = "Fig. 6 - Monthly Incidents by Borough", 
    x = "Month",
    y = "Total Incidents"
  ) +
  scale_x_date(date_labels = "%b %Y", date_breaks = "6 month") +
  theme(axis.text.x = element_text(angle = 45, hjust = 1))


# Plot incidents by year for each borough with side-by-side bars
ggplot(monthly_totals_by_borough, aes(
  x = year(month), 
  y = monthly_incidents, 
  fill = BORO)
  ) +
  # Use dodge for side-by-side bars
  geom_bar(stat = "identity", position = "dodge") +
  labs(
    title = "Fig. 7 - Yearly Incidents by Borough",
    x = "Year",
    y = "Total Incidents")


Fig 6/7 -  Boroughs over time - interesting to see that not all boroughs follow the same trends over time, and that the first covid spike was driven heavily by increased in Brooklyn.

---


In [ ]:

# plot for total victims of each sex by perpetrator sex
ggplot(totals_by_sex, aes(
  x = PERP_SEX, 
  y = Total_Victims, 
  fill = VIC_SEX)
  ) +
  geom_bar(stat = "identity", position = "dodge") +
  labs(
    title = "Fig. 8 - Victim Sex by Perpetrator Sex",
    x = "Perpetrator Sex",
    y = "Total Victims",
    fill = "Victim Sex"
  ) +
  theme_minimal()


Fig 8 - Totals of victim sex group by perpetrator sex - there are fewer total female victims than I would have expected. I'm going to use this visualization as inspiration for my first simple model below.

We could keep going with similar visuals (breakdown by race, gender, age group, etc. or relationships between group like victime age relative to perpetrator age) but I'll stop there. I'm going to focus on victim sex for the analysis and modeling component. I want to see how predictive of a victime being female some of the other attributes (perpetrator race and sex, victime race). I'll start with a simple model of victim sex as predicted by perpetrator sex. To start will have to exclude the "unknowns" from victim sex and clean up some of the other factors.


---

## Modeling

### Sex differences among victims and perpetrators

For my first simple model I want to do a basic breakdown or how a perpetrator of a given sex affects the odds of the victim being a give sex.

In [ ]:
# Convert all categorical variables to factors
clean_incident_df$VIC_SEX <- factor(clean_incident_df$VIC_SEX)
clean_incident_df$BORO <- factor(clean_incident_df$BORO)
clean_incident_df$VIC_RACE <- factor(clean_incident_df$VIC_RACE)
clean_incident_df$PERP_RACE <- factor(clean_incident_df$PERP_RACE)
clean_incident_df$PERP_SEX <- factor(clean_incident_df$PERP_SEX)

# Apply droplevels to all factor columns (to remove unused levels)
clean_incident_df <- clean_incident_df %>%
  mutate_if(is.factor, droplevels)

# Set "M" as the victim reference so that we model the odds of being "Female"
# Set the reference race for victims and perps as "white"
clean_incident_df$VIC_SEX <- relevel(clean_incident_df$VIC_SEX, ref = "M")
clean_incident_df$VIC_RACE <- relevel(clean_incident_df$VIC_RACE, ref = "WHITE")
clean_incident_df$PERP_RACE <- relevel(clean_incident_df$PERP_RACE, ref = "WHITE")

# Check the levels of the factor to confirm they are correct
levels(clean_incident_df$VIC_SEX)
levels(clean_incident_df$PERP_SEX)
levels(clean_incident_df$PERP_RACE)

In [ ]:
# create a simple generalize linear model to predict odds of female victim
# based on perp sex
simple_model_vic_sex <- glm(VIC_SEX ~ PERP_SEX,
  family=binomial,
  data=clean_incident_df,
  na.action = na.exclude)

summary(simple_model_vic_sex)

It looks like each perpetrator sex option has a statistically significant effect on the change in log-odds of the victim being female, so I'll pull out each of them and convert them to odds.

In [ ]:
print("Log-odds are")
print(coef(simple_model_vic_sex)[c("PERP_SEXM","PERP_SEXU")])

#convert log-odds to odds and print
print("The odds relative to the female victim/female perpetrator baseline are")
print((exp(coef(simple_model_vic_sex)[c("PERP_SEXM","PERP_SEXU")])))

# Print odds for when the perpetrator is female
paste(
  "The odds of a victim being female when the perpetrator is female are",
  round(exp(coef(simple_model_vic_sex)["(Intercept)"]), 2)
)

# Print odds for when the perpetrator is male
paste0(
  "The odds of a victim being female when the perpetrator is male are ",
  round(exp(coef(simple_model_vic_sex)["PERP_SEXM"]), 2) * 100,
  "% of baseline"
)

# Print odds for when the perpetrator is unknown
paste0(
  "The odds of a victim being female when the perpetrator is unknown are ",
  round(exp(coef(simple_model_vic_sex)["PERP_SEXU"]), 2) * 100,
  "% of baseline"
)

### Simple Summer prediction from Winter data

This isn't exactly a model, but I wanted to see how consistently the ratios of incidents in the Summer relative to Winter have been year over year.

In [ ]:
# make a better df for monthly modeling
monthly_totals <- time_series_df %>%
  group_by(month_year) %>%
  summarise(total_by_month = sum(total_by_day))

# Check the aggregated data
tail(monthly_totals)

In [ ]:
# make a simple ratio model based on high and low seasons
# Aggregate by year and calculate sum for January-March and 
seasonal_totals <- df_aggregated %>%
  mutate(season = case_when(
    month %in% c(1, 2, 3) ~ "low_season",
    month %in% c(6, 7, 8) ~ "high_season",
    TRUE ~ NA_character_
  )) %>%
  filter(!is.na(season)) %>%  # Filter to keep only rows with season
  group_by(year, season) %>%
  summarise(total = sum(total_by_month, na.rm = TRUE), .groups = "drop") %>%
  pivot_wider(names_from = season, values_from = total) %>%
  mutate(ratio = (high_season / low_season) * 100)

# plot the simple ratio results
ggplot(seasonal_totals, aes(
  x = year,
  y = ratio
)) + 
  geom_bar(
    stat = "identity",
    fill = "skyblue"
  ) + 
labs(
  title = "High Season (Jun-Aug) vs Low season (Jan-Mar)",
  x = "Year",
  y = "% of Low Season"
)

I think this is an interesting view of how consistently the ratio of incidents in the worst 3 months each year are to the lowest 3 months. This is an example of a quick and easy naive estimation that could be used for things like predicting staffing or resource needs for later in the year.

### More advance forecasting model


I want to take it a step further and see what could be done with some simple forecasting models. Will use seasonal ARIMA (SARIMA) which considers seasonality as well as recent trends to create a future forecast. It's import to not that this model uses only the previous data, it doesn't consider other variables. For instance, if there are always higher shooting rates when the year before was hotter or the stock market was lower, it won't include that information.

In [ ]:
# make time series friendly
start_year <- as.numeric(format(min(monthly_totals$month_year), "%Y"))
start_month <- as.numeric(format(min(monthly_totals$month_year), "%m"))

# Convert to a time series object (monthly frequency)
monthly_incident_ts <- ts(monthly_totals$total_by_month, start = c(start_year, start_month), frequency = 12)

# Check the time series
print(monthly_incident_ts)

In [ ]:
sarima_forecast <- auto.arima(monthly_incident_ts, seasonal = TRUE)

# Summary of the SARIMA model
summary(sarima_forecast)

# Forecasting the next 12 months
forecast_values <- forecast::forecast(sarima_forecast, h = 12)

# Plot the forecast
autoplot(forecast_values) +
  labs(x = "Date", y = "Value", title = "SARIMA Forecast") +
  theme(axis.text.x = element_text(angle = 45, hjust = 1))

# create a zoomed in version
autoplot(forecast_values) +
  labs(title = "Zoomed Forecast", x = "Year", y = "Totals") +
  xlim(c(2019, 2025)) +  # Adjust x-axis to show the previous 12 months and forecast
  theme(axis.text.x = element_text(angle = 45, hjust = 1))

In [ ]:
# Print the confidence intervals for the next 5 months
next_12_conf_intervals <- data.frame(
  month2024 = c(1:12),  
  forecast = forecast_values$mean[1:12],
  lower_80 = forecast_values$lower[1:12, 1],  # 80% lower bound
  upper_80 = forecast_values$upper[1:12, 1],  # 80% upper bound
  lower_95 = forecast_values$lower[1:12, 2],
  upper_95 = forecast_values$upper[1:12, 2]
) %>%
  mutate(across(everything(), \(x) round(x, 0))
)

next_12_conf_intervals

That's pretty neat. Relatively wide confidence intervals but the forecasted trend looks reasonable and could be useful for relatively low effort.

That's the end of the analysis I'm comfortable with. Below I wanted to see what it would look like to do similar modeling with multiple predictor variables (perp sex, race, borough.) It looks like it worked, but it gets out of hand to interpret it pretty quickly so I just stopped and left it here as an interesting example of what else could be done.

In [ ]:
model_vic_sex <- glm(VIC_SEX ~ BORO + VIC_RACE + 
  PERP_SEX + PERP_RACE, 
  family=binomial, 
  data=clean_incident_df,
  na.action = na.exclude)

print(levels(clean_incident_df$VIC_RACE))
print(levels(clean_incident_df$PERP_RACE))
print(levels(clean_incident_df$BORO))
print(summary(model_vic_sex))

In [ ]:
#| echo: false
#| include: false

library(broom)

tidy_model <- tidy(model_vic_sex)
print(tidy_model, n=Inf)

In [ ]:
#| echo: false
#| include: false

# Convert to a dataframe and print
tidy_model_df <- as.data.frame(tidy_model)

# Print the dataframe (all rows will be printed)
# Format specific numeric columns with decimal notation and fixed decimal places
tidy_model_df_formatted <- tidy_model_df
tidy_model_df_formatted$p.value <- format(
  tidy_model_df_formatted$p.value, scientific = FALSE, digits = 1)


# Print the entire dataframe with the formatted column(s)
(tidy_model_df_formatted)

## Potential Bias {.smaller}

There are several sources of potential bias in this report:

* **Collection bias**: Some of the categorical variable options, such as the descriptions of locations or location categories, are limited, and some actual locations may be ambiguous. The racial categorization options are also somewhat limited compared to the true diversity of racial and ethnic backgrounds. The racial categorization is likely to have been chosen by someone other than the subject, so the selected race may not be accurate or reflect how the subject (victim or perpetrator) would self-identify.

* **Analysis bias**: There are likely many interesting insights in this data regarding race (relationship between race of perpetrator and victim, racial distribution of incidents throughout the boroughs, etc.), but I deliberately avoided any in-depth analysis of those categories. I believed it would be more complicated and nuanced than my current ability and available time would allow me to do well.
---

In [ ]:
sessionInfo()